# PyTorch Tutorial - Sequence Models

In view of the AI quiz, capstone and other deliverables during week 5, the exercises for this lab is made relatively straightforward. The homework exercises can be found at the end of this tutorial.

## Sentiment detection with BERT

The task tackled here is sentiment detection of IMDB articles. In this notebook, we will utilize the Huggingface suite of libraries (transformers, datasets, evaluate, etc) to build upon it to perform some quick prototyping and evaluation of models. Some of the concepts we will use:
- loading pre-trained models
- fine-tuning on downstream tasks
- streamlining the training loop
- evaluation

In [ ]:
import torch
torch.cuda.is_available()

First, we install and import various libraries that are useful for this task, namely:
- datasets: Contains numerous popular datasets of different modalities and sizes, along with useful pre-processing tools (https://github.com/huggingface/datasets)
- transformers: Contains numerous pre-trained model from the huggingface community, along with many useful pipelines for automating the training and evaluation process (https://github.com/huggingface/transformers)
- evaluate: Contains numerous standard and custom evaluation metrics that can be easily used as part of the evaluation pipeline (https://github.com/huggingface/evaluate)

In [ ]:
!pip install -U evaluate
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers

import numpy as np
import pandas as pd
import evaluate
import accelerate
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer



The evaluate library provides many standard evaluation metrics (such as f1, precision, accuracy, etc) for specific tasks. For example, the list of standard metrics for the text classification task can be found at https://huggingface.co/tasks/text-classification. Similarly, there are also many custom-made metrics for specific tasks, which the foillowing code prints out.

In [ ]:
metrics_list = evaluate.list_evaluation_modules()
print(metrics_list)

We now load in the dataset of interest for this task, which is the IMDB dataset that contains a series of movie reviews and the corresponding sentiment label.

In [ ]:
dataset = load_dataset("imdb")

In [ ]:
print(dataset)

##Transfer Learning

Transfer learning is technique where we take a model that has learnt to solve one problem, then apply it to solving a different but related problem. In this tutorial, we are applying transfer learning by taking the pre-trained BERT-tiny model and fine-tuning it on this task of sentiment detection on IMDB. Details on the BERT-tiny model can be found at https://huggingface.co/google/bert_uncased_L-2_H-128_A-2.

The following code loads a model checkpoint from huggingface (i.e., BERT-tiny as defined in the model_checkpoint variable), which you can use to further fine-tune on the specific downstream task (i.e., IMDB sentiment detection). Apart from a standard huggingface model checkpoint, you can also use this to load one of your previously trained model, e.g., to continue more training or transfer learn on another task. This code also performs some pre-processing of the dataset in terms of tokenization and padding/truncating the text to a specific sequence that BERT requires.

In [ ]:
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(input_data):
    texts = input_data
    return tokenizer(texts["text"], max_length=max_len, padding="max_length", truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset['train']

AutoModelForSequenceClassification provides a high-level interface/wrapper around specific pre-trained models that you can use for general text classification tasks, including finetuning it based on various configurations. More details can be found at https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification.

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1)
model.resize_token_embeddings(len(tokenizer)) # need to resize due to new tokens added

Here, TrainingArguments allow us to specific various parameters relating to the training of our model, including training epochs, batch sizes, etc. More details can be found at https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments.

In [ ]:
metric_name = 'f1'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

For computation of our evaluation metric

In [ ]:
metric = evaluate.load(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")

Now, we initialize a Trainer that handles the training loop based on our definition of the model and training parameters as defined in the earlier part of this tutorial.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Start the actual model training.

In [ ]:
train_log = trainer.train()

In [ ]:
# trainer.save_model("./models/myFinetunedModel") # for saving your model

Finally, we perform our evaluation on our test set using the fine-tuned model from earlier.

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")
results = classifier(dataset['test']['text'], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')
f1 = metric.compute(predictions=dfResults['label'].tolist(), references=dataset['test']['label'], average='micro')
print(f1)

## Homework Exercises
**Due: 1st Mar, 11:59pm**
<br>
<br>
Submit your homework as either: (i) an ipynb file with your results inside; or (ii) a python file and separate pdf discussing your results. Based on what you have done so far in this tutorial, repeat another set of experiments with the following changes:

(a) There is a potential error in the design of the training and testing/evaluation step in this tutorial. Identify what it is and describe how you will solve this.

(b) Pick another dataset from the datasets library that is not a binary classification task (see https://huggingface.co/datasets). You can sample a subset from this dataset to reduce compute time (see https://huggingface.co/docs/datasets/en/process).

(c) Fine-tune the same model but with a dropout of 0.2, over 3 epochs and report on accuracy scores, in addition to F1 scores.

